<a href="https://colab.research.google.com/github/AchrafAsh/gnn-receptive-fields/blob/main/05_measuring_problem_radius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
try:
    os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
except:
    pass
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [3]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

from collections import Counter
from torch_geometric.utils import to_dense_adj
from tqdm.notebook import tqdm

%matplotlib inline
sns.set_style('darkgrid')

In [4]:
def get_k_hop_neighborhood(k:int, adj: torch.tensor) -> list:
    """Returns the list of k-hop dense adjacency matrices
    
    Args:
        - k (int): maximum depth of neighborhood to compute
        - adj [num_nodes, num_nodes]: dense adjacency matrix
    
    :rtype: List[torch.Tensor]
    """
    
    k_hop_neb = [adj.clone()]
    pow_A = adj.clone()

    for l in tqdm(range(k-1)):
        pow_A = torch.mm(adj, pow_A)
        k_neb = torch.where(
            torch.where(pow_A > 0, 1, 0) - sum(k_hop_neb) > 0,
            1,
            0
        )
        k_hop_neb.append(k_neb)
    
    return k_hop_neb

In [5]:
def scale(X:torch.Tensor):
    """Returns the scaled features of the graph

    Args:
        - X [num_nodes, num_features]
    """
   
    m = torch.mean(X, dim=0)
    s = torch.std(X, dim=0)
    return (X - m)/ s

In [6]:
def centroids(X:torch.Tensor, y:torch.Tensor):
    """Returns the label representation by averaging its nodes' features

    Args:
        - X [num_nodes, num_features]: node features
        - y [num_nodes]: labels
    """
    
    # group nodes by label
    obs = {}
    for i in range(X.size(0)):
        if obs.get(y[i].item()):
            obs[y[i].item()] += [X[i]]
        else:
            obs[y[i].item()] = [X[i]]

    avg_obs = [ sum(obs[c]) / len(obs[c]) for c in range(7) ]
    
    # convert to torch tensor
    return torch.tensor(list(map(lambda x: x.numpy(), avg_obs)))

In [32]:
def feature_correlation(all_adj, X, barycenters, k):
    """Returns the list of correlations between the barycenter representation of
    labels and the neighbor features.

    Args:
        - all_adj: list of k-hop adjacency dense matrices [num_nodes, num_nodes]
        - X: [num_nodes, num_features] features
        - barycenters: [num_classes, num_features]
        - k (int): depth of the neighborhood

    :rtype: list - [num_nodes]
    """

    return [np.mean(np.corrcoef(
                x=X[ sum(all_adj[:k])[i] == 1 ].numpy(),
                # x=X[ all_adj[k-1][i] == 1 ].numpy(),
                y=barycenters[G.y[i].item()].numpy()
            )[:-1, -1].abs()) if (all_adj[k-1][i] == 1).sum() > 0 else 0
            for i in range(X.size(0))]

In [29]:
# TODO: homophily index -- compute the average of all the nodes in the graph
def homophily_index(neb:torch.tensor, mask=None):
    """Computes the homophily index for a given depth
    
    TODO - compute the average of all the nodes

    Args:
        - neb [num_nodes, num_nodes]: neighbors to consider (can be the adjacency matrix or k-hop neighborhood)
        - mask [num_nodes]: "train", "test" or "val" to consider only these specific neighbors
    """
    
    if mask == None:
        mask = torch.ones(G.num_nodes)

    # neb_labels = G.y[neb[i] * mask == 1] # looking at the labels of the neighbors (within the mask)
    # if len(neb_labels) == 0: return 0
    # return (labels == G.y[i]).sum().item() / len(labels)
    
    return np.mean([(G.y[neb[i] * mask == 1] == G.y[i]).float().mean().item() if (neb[i] * mask == 1).sum() > 0 else 0 for i in range(G.num_nodes)])

In [31]:
homophily_index(to_dense_adj(G.edge_index).squeeze(0), G.train_mask)

0.8251578295204453

## Test with different datasets

In [2]:
%%capture
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py

from data import load_dataset

path = osp.join(os.getcwd(), 'data')
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0] # only graph of the dataset